In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# 데이터 로딩 및 전처리
df = pd.read_csv('abalone.csv')
df = df.drop(columns=['id'])
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])

X = df.drop(columns=['Rings']).values
y = pd.cut(df['Rings'], bins=[0, 8, 10, 30], labels=[0, 1, 2]).astype(int).values  # 0=young, 1=adult, 2=old

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dataset 정의
class AbaloneClassificationDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(AbaloneClassificationDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(AbaloneClassificationDataset(X_test, y_test), batch_size=64)

# 모델 정의
class ClassificationModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ClassificationModel(X.shape[1], 3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
for epoch in range(30):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"[분류] Epoch {epoch+1}/30, Loss: {loss.item():.4f}")

# 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model(X_batch)
        predicted = torch.argmax(preds, dim=1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)
print(f"🎯 분류 정확도: {correct / total * 100:.2f}%")


[분류] Epoch 1/30, Loss: 0.9643
[분류] Epoch 2/30, Loss: 1.1065
[분류] Epoch 3/30, Loss: 0.7382
[분류] Epoch 4/30, Loss: 0.6047
[분류] Epoch 5/30, Loss: 0.6734
[분류] Epoch 6/30, Loss: 0.5900
[분류] Epoch 7/30, Loss: 0.6198
[분류] Epoch 8/30, Loss: 0.7907
[분류] Epoch 9/30, Loss: 0.6102
[분류] Epoch 10/30, Loss: 0.6513
[분류] Epoch 11/30, Loss: 0.6430
[분류] Epoch 12/30, Loss: 0.8747
[분류] Epoch 13/30, Loss: 0.4912
[분류] Epoch 14/30, Loss: 0.8111
[분류] Epoch 15/30, Loss: 0.7373
[분류] Epoch 16/30, Loss: 0.6661
[분류] Epoch 17/30, Loss: 0.9329
[분류] Epoch 18/30, Loss: 0.4054
[분류] Epoch 19/30, Loss: 0.8898
[분류] Epoch 20/30, Loss: 0.7745
[분류] Epoch 21/30, Loss: 0.4355
[분류] Epoch 22/30, Loss: 0.7119
[분류] Epoch 23/30, Loss: 0.7186
[분류] Epoch 24/30, Loss: 0.6956
[분류] Epoch 25/30, Loss: 0.7239
[분류] Epoch 26/30, Loss: 0.9339
[분류] Epoch 27/30, Loss: 0.7680
[분류] Epoch 28/30, Loss: 0.9100
[분류] Epoch 29/30, Loss: 0.7660
[분류] Epoch 30/30, Loss: 0.7065
🎯 분류 정확도: 65.07%
